In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os 
import sys

os.chdir(r'C:\Users\Asus\Documents\Holmusk data\datasciencepositionatholmusk')
bill_amount = pd.read_csv('bill_amount.csv')
bill_id = pd.read_csv('bill_id.csv')
clinical_data = pd.read_csv('clinical_data.csv')
demo = pd.read_csv('demographics.csv')


In [2]:
clinical_data = clinical_data.sort_values(['id', 'date_of_admission'])

In [3]:
cd_dup = clinical_data[clinical_data.duplicated(subset = ['id'], keep = False)]

In [4]:
cd_dup.medical_history_3 = cd_dup.medical_history_3.apply(lambda x: x if type(x) == int else (1 if x == 'Yes' else (0)))

C:\Users\Asus\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
cd_dup = cd_dup.fillna(value = 0)

In [6]:
cd_dup.medical_history_2 = cd_dup.medical_history_2.astype('int64')
cd_dup.medical_history_5 = cd_dup.medical_history_5.astype('int64')

In [7]:
cd_dup.date_of_admission = cd_dup.date_of_admission.astype('datetime64')
cd_dup.date_of_discharge = cd_dup.date_of_discharge.astype('datetime64')

In [8]:
dg = cd_dup.groupby('id')

In [9]:
dg.date_of_admission = dg.date_of_admission.apply(lambda x: x.astype('datetime64'))

In [10]:
Admission_interval = []
patient_id_key = []
for idx, group in dg:
    for i in range(len(group['date_of_admission']) - 1):
        Admission_interval.append(np.datetime64(group.date_of_admission.iloc[i +1]) - np.datetime64(group.date_of_admission.iloc[i]))
        patient_id_key.append(idx)

In [11]:
a = list(zip(Admission_interval, patient_id_key))

In [12]:
Adm_int = pd.DataFrame.from_records(a, columns = ['Ad_int', 'Patient_ID']).set_index('Patient_ID')

In [13]:
Adm_int.Ad_int = Adm_int.Ad_int.dt.days

In [14]:
Readmissions_in_30D = Adm_int[Adm_int.Ad_int < 30]

In [15]:
Readmissions_in_30D

,Ad_int
Patient_ID,
0a90cbac6388ed9938799cecc49015e2,26
371aef4418fe5702448f2c943cac88c9,22
4e46fddfa404b306809c350aecbf0f6a,17
53c417b05a13eaf7afc77cc8664bab4c,25
61cdcabfb2f527da991be05ee39c15a6,25
738a6a6c8cd797d456de53754cdb4750,19
9ca72062a5f6b74a36dd7c7bf447dd62,9
b38d3fd6a9e45f1fb8fdbd137027fbb2,11
c237b1af83870ddcb5c74a42f23a1131,12


In [16]:
Abnormal_history = []
patient_id_key_2 = []
for idx, group in dg:
    for i in range(len(group['date_of_admission']) - 1):
        if group.medical_history_1.iloc[i] > group.medical_history_1.iloc[i + 1]:
            Abnormal_history.append('1')
            patient_id_key_2.append(idx)
        if group.medical_history_2.iloc[i] > group.medical_history_2.iloc[i + 1]:
            Abnormal_history.append('2')
            patient_id_key_2.append(idx)
        if group.medical_history_3.iloc[i] > group.medical_history_3.iloc[i + 1]:
            Abnormal_history.append('3')
            patient_id_key_2.append(idx)
        if group.medical_history_4.iloc[i] > group.medical_history_4.iloc[i + 1]:
            Abnormal_history.append('4')
            patient_id_key_2.append(idx)
        if group.medical_history_5.iloc[i] > group.medical_history_5.iloc[i + 1]:
            Abnormal_history.append('5')
            patient_id_key_2.append(idx)
        if group.medical_history_6.iloc[i] > group.medical_history_6.iloc[i + 1]:
            Abnormal_history.append('6')
            patient_id_key_2.append(idx)
        if group.medical_history_7.iloc[i] > group.medical_history_7.iloc[i + 1]:
            Abnormal_history.append('7')
            patient_id_key_2.append(idx)

In [17]:
ab_hist = list(zip(Abnormal_history, patient_id_key_2))

In [18]:
Abn_Hist = pd.DataFrame.from_records(ab_hist, columns = ['Abnormal_history', 'Patient ID']).set_index('Patient ID')

In [19]:
Patients_w_Abn_Hist = list(set(patient_id_key_2))

In [20]:
len(Patients_w_Abn_Hist)

221

In [22]:
cd_dup = cd_dup.set_index('id')

In [23]:
P_w_A_H = cd_dup.loc[Patients_w_Abn_Hist,:]

In [24]:
P_w_A_H

,date_of_admission,date_of_discharge,medical_history_1,medical_history_2,medical_history_3,medical_history_4,medical_history_5,medical_history_6,medical_history_7,preop_medication_1,...,symptom_1,symptom_2,symptom_3,symptom_4,symptom_5,lab_result_1,lab_result_2,lab_result_3,weight,height
id,,,,,,,,,,,,,,,,,,,,,
f2c575f83f128df84aba17276fc85b39,2011-01-24,2011-02-03,0,1,0,0,0,0,0,1,...,1,1,1,1,1,13.3,32.8,120.0,104.2,173.0
f2c575f83f128df84aba17276fc85b39,2014-04-15,2014-04-22,0,0,0,0,0,0,1,0,...,0,1,0,1,1,12.5,23.8,61.0,101.2,173.0
b7c431cf57fb6a2a5995dbc0774afa0f,2014-08-21,2014-08-28,0,0,0,0,1,1,0,0,...,1,1,0,1,1,15.1,26.7,87.0,82.6,174.0
b7c431cf57fb6a2a5995dbc0774afa0f,2014-12-14,2014-12-23,0,1,0,0,0,0,0,0,...,0,0,1,1,0,16.6,31.6,110.0,80.6,174.0
0a90cbac6388ed9938799cecc49015e2,2011-06-26,2011-07-05,1,0,1,0,0,0,0,0,...,1,1,1,0,0,16.2,28.1,106.0,77.6,160.0
0a90cbac6388ed9938799cecc49015e2,2011-07-22,2011-08-02,0,1,0,0,0,0,1,0,...,1,1,0,1,1,14.1,29.3,116.0,78.6,160.0
0ec2b2812e69df2ede31b5ace2599db5,2012-05-02,2012-05-09,0,1,0,0,1,0,0,1,...,1,0,1,1,0,15.5,30.6,103.0,85.8,179.0
0ec2b2812e69df2ede31b5ace2599db5,2015-10-19,2015-10-30,0,0,0,0,0,1,0,1,...,0,1,1,1,1,13.1,26.7,79.0,84.8,179.0
3566fa57efe0ca8c6c90d94c65fef821,2012-09-19,2012-10-06,0,1,0,0,0,1,0,1,...,0,1,1,0,0,18.0,25.3,88.0,67.0,159.0
